In [91]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [92]:
# load YAML
with open('data.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)

['mobile(E-waste)', 'E-waste', 'plastic(Non E waste)', 'keyboard(E-waste)', 'laptop(E-waste)', 'headphones(E-waste)', 'television(E-waste)', 'mouse(E-waste)', 'battery(E-waste)', 'printer(E-waste)', 'charger(E-waste)', 'microwave(E-waste)', 'vegetable_peel(not a E-waste)', 'paper(Non E-waste)', 'fruit(not a E-waste)', 'egg(not a E-waste)', 'vegetablel(not a E-waste)', 'food(not a E-waste)', 'wires(E-waste)', 'paper(not a E-waste)']


In [93]:
#loading yolo model
yolo = cv2.dnn.readNetFromONNX('./Model-20240415T134521Z-001/Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [94]:
# loading the image
img = cv2.imread('nokia.jpeg')
image = img.copy()
row, col, d = image.shape


# getting the YOLO prediction from the the image
# step-1 converting image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image
# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from YOLO

In [95]:
print(preds.shape)

(1, 25200, 25)


In [96]:


detections = preds[0]
boxes = []
confidences = []
classes = []

# width and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w / INPUT_WH_YOLO
y_factor = image_h / INPUT_WH_YOLO

# Iterate through detections
for detection in detections:
    confidence = detection[4]  # Confidence of detecting an object
    if confidence > 0.1:  # Adjust the confidence threshold as needed
        class_scores = detection[5:] if len(detection) >= 6 else []
        if len(class_scores) > 0:  # Check if class_scores has any elements
            class_score = np.max(class_scores)
            class_id = np.argmax(class_scores)
            if class_score > 0.25:
                cx, cy, w, h = detection[0:4]
                # Construct bounding box from four values: left, top, width, and height
                left = int((cx - 0.5 * w) * x_factor)
                top = int((cy - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = [left, top, width, height]  # No need to convert to numpy array
                # Append values into the lists
                confidences.append(float(confidence))  # Ensure confidence is float
                boxes.append(box)
                classes.append(class_id)

# Converting lists to numpy arrays
boxes_np = np.array(boxes)
confidences_np = np.array(confidences)

# Non-Maximum Suppression (NMS)
if len(boxes_np) > 0:
    indices = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.tolist(), 0.25, 0.45)
    indices = indices.flatten()

    # Drawing bounding boxes and printing names of detected objects
    for ind in indices:
        if ind < len(classes):
            # Extract bounding box
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            class_id = classes[ind]

            # Check if the class_id is within the range of labels list
            if class_id < len(labels):
                class_name = labels[class_id]
                # Print the name of the detected object
                print("Detected:", class_name)

                text = f'{class_name}: {bb_conf}%'

                cv2.rectangle(input_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.rectangle(input_image, (x, y - 30), (x + w, y), (255, 255, 255), -1)

                cv2.putText(input_image, text, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
            else:
                print(f"Class ID {class_id} is out of range for labels list")
        else:
            print(f"Index {ind} is out of range for classes list")

# Show the image with bounding boxes and class names
cv2.imshow("Image", input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Detected: mobile(E-waste)
